Realizamos los imports necesarios e iniciamos una SparkSession

In [2]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` 
import $ivy.`sh.almond::almond-spark:0.4.0`

import org.apache.spark.sql.{NotebookSparkSession, SparkSession}
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._

val spark = NotebookSparkSession
      .builder()
      .config("spark.sql.join.preferSortMergeJoin", false)
      .config("spark.sql.shuffle.partitions", 64)
      .master("local[4]")
      .getOrCreate()

import spark.implicits._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

Logger.getRootLogger().setLevel(Level.ERROR)

Creating SparkSession


Spark UI

import $ivy.$                                   

import $ivy.$                              


import org.apache.spark.sql.{NotebookSparkSession, SparkSession}

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.types._


spark: SparkSession = org.apache.spark.sql.SparkSession@1ccf6208
import spark.implicits._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}



In [ ]:
val data: DataFrame = spark.read.option("multiline", "true").json("D:/TFGAlvaroSanchez/data2/*.json")

In [ ]:
data.show()

### Obtencion de la fecha y del valor maximo de temperatura

Con lo siguiente cambiamos el tipo de datos de la columna ta_max

val data1 = data.withColumn("ta_max", $"ta_max".substr(0,4).cast(IntegerType))

In [ ]:
import org.apache.spark.sql.functions.{min, max, desc}

data.withColumn("ta_max", $"ta_max".substr(0,4).cast(IntegerType))
    .select($"fecha", $"ta_max" as "temperatura maxima")
    .orderBy($"ta_max".desc)
    .show(1)

### Obtencion del mes, año, id estacion y provincia donde ocurren la stemperaturas mas altas en 2021

In [ ]:
val data2021 : DataFrame = spark.read.option("multiline", "true").json("D:/TFGAlvaroSanchez/data2/*2021.json")

In [ ]:
data2021.printSchema()

In [ ]:
data2021.select($"ta_min", $"indicativo", $"fecha")
    .filter($"indicativo".equalTo("2916A"))
    .show()

Usamos `.df` para darle nombre a las columnas

In [ ]:
val ids : DataFrame = spark.read.option("delimiter", ";").csv("D:/TFGAlvaroSanchez/data2/aemetID.csv")
    .toDF("provincia", "indicativo", "ubicacion")

In [ ]:
ids.show()
ids.printSchema()

In [ ]:
val joindata : DataFrame = data2021.join(ids, "indicativo")

In [ ]:
joindata.withColumn("ta_max", $"ta_max".substr(0,4).cast(IntegerType))
    .select($"fecha".substr(6,2) as "mes", $"fecha".substr(0,4) as "año", $"ta_max" as "temperatura maxima", $"indicativo", $"provincia")
    .filter(!($"mes".equalTo("13")))
    .orderBy($"ta_max".desc)
    .show()

### Obtencion del mes, año y provincia donde se encuntra la temperatura mas baja durante verano (junio, julio y agosto)

In [ ]:
joindata.withColumn("ta_min", $"ta_min".substr(0,4).cast(IntegerType))
    .withColumn("fecha", $"fecha".cast(DateType)).printSchema

Para quedarnos unicamente con la temperatura ya que este valor viene precedido del dia que ocurrio de la siguiente manera: temperatura(dia)

In [ ]:
import org.apache.spark.sql.functions.split

joindata.withColumn("ta_min", split($"ta_min", "\\("))
    .select($"ta_min"(0) as "val")
    .show()

In [ ]:
joindata.withColumn("ta_min", split($"ta_min", "\\("))
    .withColumn("ta_min", $"ta_min"(0).cast(IntegerType))
    .withColumn("fecha", $"fecha".cast(DateType))
    .select(func.month($"fecha") as "mes", func.year($"fecha") as "año", $"ta_min" as "temperatura minima", $"provincia")  
    .filter($"mes" > 5 && $"mes" < 9)
    .orderBy($"temperatura minima".asc)
    .show()

### Numero de meses con temperaturas maximas >30º por provincia

In [ ]:
joindata
    .withColumn("ta_max", split($"ta_max", "\\("))
    .withColumn("ta_max", $"ta_max"(0).cast(IntegerType))
    .withColumn("fecha", $"fecha".cast(DateType))
    .filter($"ta_max" > 30 && !func.isnull($"fecha"))//> "0000-00-00")
    .groupBy("provincia")
    .count
    .show()

Relacion que tiene el calor con los meses de verano

In [ ]:
joindata
    .withColumn("ta_max", split($"ta_max", "\\("))
    .withColumn("ta_max", $"ta_max"(0).cast(IntegerType))
    .withColumn("fecha", $"fecha".cast(DateType))
    .filter($"ta_max" > 30 && !func.isnull($"fecha"))
    .withColumn("mes", func.month($"fecha"))
    .stat.corr("ta_max", "mes")

### Uso de graficos mostrando la precipitacion mensual por meses y años en Vitigudino(Salamanca)

In [ ]:
val vitData : DataFrame = spark.read.option("multiline", "true").json("D:/TFGAlvaroSanchez/data2/2916A*.json")

Obtenemos los datos que queremos mostrar

In [ ]:
val vitDataRequired : DataFrame = vitData
    .withColumn("fecha", $"fecha".cast(DateType))
    .withColumn("p_mes", $"p_mes".cast(IntegerType))
    .filter(!func.isnull($"fecha") && $"p_mes" >= 0)
    .select($"fecha", $"p_mes")
    .orderBy($"fecha".asc)

In [ ]:
vitDataRequired.show()

In [ ]:
val vitDataRequired_f : Seq[String] = vitDataRequired.select($"fecha").as[String].collect.toSeq
val vitDataRequired_p : Seq[Int] = vitDataRequired.select($"p_mes").as[Int].collect.toSeq

Realizamos los imports necesarios para las graficas

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

In [ ]:
val trace = Seq(
    Scatter(
        vitDataRequired_f,
        vitDataRequired_p,
        fill = Fill.ToZeroY,
        marker = Marker(
            color = Color.RGBA(55, 128, 191, 0.6)
        )
    )
)


val layout = Layout(
    title = "Precipitacion mensual (mm)",
    paper_bgcolor =  Color.RGBA(245, 246, 249, 1),
    plot_bgcolor = Color.RGBA(245, 246, 249, 1),
)
plot(trace, layout)

### Leer los datos de manera casteada mediante un schema

In [ ]:
val schema = StructType(
                Array(
                    /*StructField("fecha", DateType, true),
                    StructField("indicativo", StringType, true),
                    StructField("p_max", StringType, true),*/
                    StructField("glo", IntegerType, true),
                    /*StructField("hr", IntegerType, true),
                    StructField("nw_55", IntegerType, true),
                    StructField("tm_min", DoubleType, true),
                    StructField("ta_max", StringType, true),
                    StructField("ts_min", DoubleType, true),
                    StructField("nt_30", IntegerType, true),
                    StructField("n_des", IntegerType, true),
                    StructField("w_racha", StringType, true),
                    StructField("np_100", IntegerType, true)*/
                )
            )

In [ ]:
val dt: DataFrame = spark.read.schema(schema)
    .option("multiline", "true")
    .option("dateFormat", "yyyy-MM")
    .json("D:/TFGAlvaroSanchez/data2/0201D(Barcelona)-2021.json")

In [ ]:
dt.show()

In [ ]:
val dt: DataFrame = spark.read
    .option("multiline", "true")
    .json("D:/TFGAlvaroSanchez/data2/0201D(Barcelona)-2021.json")
    .select(//$"fecha".cast(DateType), 
            func.to_date($"fecha").alias("fecha"),
            $"indicativo", 
            $"p_max",
            $"glo".cast(DoubleType), 
            $"hr".cast(DoubleType), 
            $"nw_55".cast(IntegerType), 
            $"tm_min".cast(DoubleType), 
            $"ta_max", 
            $"ts_min".cast(DoubleType), 
            $"nt_30".cast(IntegerType), 
            $"n_des".cast(IntegerType), 
            $"w_racha", 
            $"np_100".cast(IntegerType), 
            $"nw_91".cast(IntegerType), 
            $"np_001".cast(IntegerType), 
            $"ta_min", 
            $"w_rec".cast(IntegerType), 
            $"e".cast(DoubleType), 
            $"np_300".cast(IntegerType), 
            $"p_mes".cast(DoubleType), 
            $"w_med".cast(DoubleType), 
            $"nt_00".cast(IntegerType), 
            $"ti_max".cast(DoubleType), 
            $"tm_mes".cast(DoubleType), 
            $"tm_max".cast(DoubleType), 
            $"np_010".cast(IntegerType))

In [ ]:
dt.printSchema()

In [ ]:
dt.show()

### Temperatura media en españa

In [ ]:
val data: DataFrame = spark.read
    .option("multiline", "true")
    .json("D:/TFGAlvaroSanchez/data2/*.json")
    .select($"fecha".cast(DateType), 
            $"indicativo", 
            $"p_max",
            $"glo".cast(DoubleType), 
            $"hr".cast(DoubleType), 
            $"nw_55".cast(IntegerType), 
            $"tm_min".cast(DoubleType), 
            $"ta_max", 
            $"ts_min".cast(DoubleType), 
            $"nt_30".cast(IntegerType), 
            $"n_des".cast(IntegerType), 
            $"w_racha", 
            $"np_100".cast(IntegerType), 
            $"nw_91".cast(IntegerType), 
            $"np_001".cast(IntegerType), 
            $"ta_min", 
            $"w_rec".cast(IntegerType), 
            $"e".cast(DoubleType), 
            $"np_300".cast(IntegerType), 
            $"p_mes".cast(DoubleType), 
            $"w_med".cast(DoubleType), 
            $"nt_00".cast(IntegerType), 
            $"ti_max".cast(DoubleType), 
            $"tm_mes".cast(DoubleType), 
            $"tm_max".cast(DoubleType), 
            $"np_010".cast(IntegerType))

In [ ]:
data.filter($"fecha".isNotNull)
    .select($"tm_mes".alias("temperatura media"))
    .describe()
    .show()

### Fecha y estaciones con temp >35º (Utilizando expr)

In [ ]:
val data: DataFrame = spark.read
    .option("multiline", "true")
    .json("D:/TFGAlvaroSanchez/data2/*.json")
    .select($"fecha".cast(DateType), 
            $"indicativo", 
            $"p_max",
            $"glo".cast(DoubleType), 
            $"hr".cast(DoubleType), 
            $"nw_55".cast(IntegerType), 
            $"tm_min".cast(DoubleType), 
            $"ta_max", 
            $"ts_min".cast(DoubleType), 
            $"nt_30".cast(IntegerType), 
            $"n_des".cast(IntegerType), 
            $"w_racha", 
            $"np_100".cast(IntegerType), 
            $"nw_91".cast(IntegerType), 
            $"np_001".cast(IntegerType), 
            $"ta_min", 
            $"w_rec".cast(IntegerType), 
            $"e".cast(DoubleType), 
            $"np_300".cast(IntegerType), 
            $"p_mes".cast(DoubleType), 
            $"w_med".cast(DoubleType), 
            $"nt_00".cast(IntegerType), 
            $"ti_max".cast(DoubleType), 
            $"tm_mes".cast(DoubleType), 
            $"tm_max".cast(DoubleType), 
            $"np_010".cast(IntegerType))
val dataWithNameStation = data.join(ids, "indicativo")

In [ ]:
dataWithNameStation.withColumn("ta_max", split($"ta_max", "\\("))
    .withColumn("t_max", $"ta_max"(0).cast(IntegerType))
    .withColumn("dia", func.substring($"ta_max"(1), 0, 2).cast(IntegerType))
    .withColumn("calor", func.expr("t_max > 35"))
    .filter($"calor" && $"fecha".isNotNull)
    .select($"ubicacion", func.year($"fecha").alias("año"), func.month($"fecha").alias("mes"), $"dia")
    .show(false)

### Media de todos los datos, agrupados por fecha y por indicativo

In [ ]:
val data2021: DataFrame = spark.read
    .option("multiline", "true")
    .json("D:/TFGAlvaroSanchez/data2/*-2021.json")
    .select($"fecha".cast(DateType), 
            $"indicativo", 
            $"p_max",
            $"glo".cast(DoubleType), 
            $"hr".cast(DoubleType), 
            $"nw_55".cast(IntegerType), 
            $"tm_min".cast(DoubleType), 
            $"ta_max", 
            $"ts_min".cast(DoubleType), 
            $"nt_30".cast(IntegerType), 
            $"n_des".cast(IntegerType), 
            $"w_racha", 
            $"np_100".cast(IntegerType), 
            $"nw_91".cast(IntegerType), 
            $"np_001".cast(IntegerType), 
            $"ta_min", 
            $"w_rec".cast(IntegerType), 
            $"e".cast(DoubleType), 
            $"np_300".cast(IntegerType), 
            $"p_mes".cast(DoubleType), 
            $"w_med".cast(DoubleType), 
            $"nt_00".cast(IntegerType), 
            $"ti_max".cast(DoubleType), 
            $"tm_mes".cast(DoubleType), 
            $"tm_max".cast(DoubleType), 
            $"np_010".cast(IntegerType))

In [ ]:
val pivotedData2021 = data2021.groupBy(func.year($"fecha")).pivot("indicativo").avg()
pivotedData2021.select("year(fecha)", "0201D_avg(tm_mes)").show()

### Leer todos los datos y transformalos a formato Parquet

In [ ]:
val allData = spark.read.option("multiline", "true").json("D:/TFGAlvaroSanchez/data/month/*.json").select(
            $"fecha".cast(DateType), 
            $"indicativo", 
            $"p_max",
            $"glo".cast(DoubleType), 
            $"hr".cast(DoubleType), 
            $"nw_55".cast(IntegerType), 
            $"tm_min".cast(DoubleType), 
            $"ta_max", 
            $"ts_min".cast(DoubleType), 
            $"nt_30".cast(IntegerType), 
            $"n_des".cast(IntegerType), 
            $"w_racha", 
            $"np_100".cast(IntegerType), 
            $"nw_91".cast(IntegerType), 
            $"np_001".cast(IntegerType), 
            $"ta_min", 
            $"w_rec".cast(IntegerType), 
            $"e".cast(DoubleType), 
            $"np_300".cast(IntegerType), 
            $"p_mes".cast(DoubleType), 
            $"w_med".cast(DoubleType), 
            $"nt_00".cast(IntegerType), 
            $"ti_max".cast(DoubleType), 
            $"tm_mes".cast(DoubleType), 
            $"tm_max".cast(DoubleType), 
            $"np_010".cast(IntegerType))

In [ ]:
allData.write.format("parquet").partitionBy("indicativo").mode("overwrite").save("D:/TFGAlvaroSanchez/data/monthParquet/")

### Otra manera de leer los datos y pasarlos a Parquet (más optima)

#### Pasar los ficheros JSON multiline a JSON line

Para esto lo que he hecho ha sido ejecutar en el CMD el siguiente comando `FOR %a IN (../data/*.json) DO jq . -c "%a" > "../JSONLine/%a"` con el que vamos recorriendo la carpeta donde se encuentran los ficheros JSON multiline y los vamos guardando con la transformacion en la carpeta JSONLine

In [ ]:
val schema = StructType(
    Array(
        StructField("fecha", StringType, true), 
        StructField("indicativo", StringType, true),
        StructField("nombre", StringType, true),
        StructField("provincia", StringType, true),
        StructField("altitud", StringType, true),
        StructField("tm_mes", StringType, true),
        StructField("tm_max", StringType, true),
        StructField("tm_min", StringType, true), 
        StructField("ta_max", StringType, true),
        StructField("ta_min", StringType, true),
        StructField("ts_min", StringType, true),
        StructField("ti_max", StringType, true),
        StructField("nt_30", StringType, true),
        StructField("nt_00", StringType, true),
        StructField("p_mes", StringType, true),
        StructField("p_max", StringType, true),
        StructField("np_001", StringType, true),
        StructField("np_010", StringType, true),
        StructField("np_100", StringType, true),
        StructField("np_300", StringType, true),
        StructField("hr", StringType, true), 
        StructField("e", StringType, true),
        StructField("n_llu", StringType, true),
        StructField("n_nie", StringType, true),
        StructField("n_gra", StringType, true),
        StructField("n_tor", StringType, true),
        StructField("n_fog", StringType, true),
        StructField("n_des", StringType, true),
        StructField("n_nub", StringType, true),
        StructField("n_cub", StringType, true),
        StructField("inso", StringType, true),
        StructField("p_sol", StringType, true),
        StructField("glo", StringType, true),
        StructField("evap", StringType, true),
        StructField("w_rec", StringType, true),
        StructField("w_racha", StringType, true),
        StructField("nw_55", StringType, true),
        StructField("nw_91", StringType, true),
        StructField("w_med", StringType, true),
        StructField("q_med", StringType, true),
        StructField("q_max", StringType, true),
        StructField("q_min", StringType, true),
        StructField("q_mar", StringType, true),
        StructField("ts_10", StringType, true),
        StructField("ts_20", StringType, true),
        StructField("ts_50", StringType, true),
        StructField("nv_0050", StringType, true),
        StructField("nv_0100", StringType, true),
        StructField("nv_1000", StringType, true)       
    )
)

In [ ]:
val allData = spark.read.schema(schema).json("D:/TFGAlvaroSanchez/data/monthJSONLine/*.json")
    .select(
            $"fecha".cast(DateType), 
            $"indicativo", 
            $"p_max",
            $"glo".cast(DoubleType), 
            $"hr".cast(DoubleType), 
            $"nw_55".cast(IntegerType), 
            $"tm_min".cast(DoubleType), 
            $"ta_max", 
            $"ts_min".cast(DoubleType), 
            $"nt_30".cast(IntegerType), 
            $"n_des".cast(IntegerType), 
            $"w_racha", 
            $"np_100".cast(IntegerType), 
            $"nw_91".cast(IntegerType), 
            $"np_001".cast(IntegerType), 
            $"ta_min", 
            $"w_rec".cast(IntegerType), 
            $"e".cast(DoubleType), 
            $"np_300".cast(IntegerType), 
            $"p_mes".cast(DoubleType), 
            $"w_med".cast(DoubleType), 
            $"nt_00".cast(IntegerType), 
            $"ti_max".cast(DoubleType), 
            $"tm_mes".cast(DoubleType), 
            $"tm_max".cast(DoubleType), 
            $"np_010".cast(IntegerType)
    )

In [ ]:
allData.write.format("parquet").partitionBy("indicativo").mode("overwrite").save("D:/TFGAlvaroSanchez/data/monthParquet/")

### Representacion de las temperaturas máximas en diferentes meses por años (Parquet sin particion)

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

In [ ]:
val dataRead = spark.read.parquet("D:/TFGAlvaroSanchez/data/monthParquet/*").na.drop()

val data = dataRead
    .withColumn("ta_max", func.split($"ta_max", "\\(")(0).cast(IntegerType))
    .groupBy($"fecha")
    .agg(func.avg($"ta_max").alias("ta_max"))
    .withColumn("año", func.year($"fecha"))
    .withColumn("mes", func.month($"fecha"))
    .orderBy("año", "mes")
    .repartition(8)

In [ ]:
def seqYear(year : Int): Seq[Double] = {
    data.filter($"año" === year).map(row => row.getDouble(1)).collect().toSeq
}

In [ ]:
val SeqMonths = data.filter($"año" === 2010).map(row => row.getInt(3)).collect().toSeq
val Seq2010 = seqYear(2010)
val Seq2011 = seqYear(2011)
val Seq2012 = seqYear(2012)
val Seq2013 = seqYear(2013)
val Seq2014 = seqYear(2014)
val Seq2015 = seqYear(2015)
val Seq2016 = seqYear(2016)
val Seq2017 = seqYear(2017)
val Seq2018 = seqYear(2018)
val Seq2019 = seqYear(2019)
val Seq2020 = seqYear(2020)
val Seq2021 = seqYear(2021)
val Seq2022 = seqYear(2022)

In [ ]:
val year2010= Scatter(
  SeqMonths,
  Seq2010,
  name = "año 2010"
)

val year2011 = Scatter(
  SeqMonths,
  Seq2011,
  name = "año 2011"
)

val year2012 = Scatter(
  SeqMonths,
  Seq2012,
  name = "año 2012"
)

val year2013 = Scatter(
  SeqMonths,
  Seq2013,
  name = "año 2013"
)

val year2014 = Scatter(
  SeqMonths,
  Seq2014,
  name = "año 2014"
)

val year2015 = Scatter(
  SeqMonths,
  Seq2015,
  name = "año 2015"
)

val year2016 = Scatter(
  SeqMonths,
  Seq2016,
  name = "año 2016"
)

val year2017 = Scatter(
  SeqMonths,
  Seq2017,
  name = "año 2017"
)

val year2018 = Scatter(
  SeqMonths,
  Seq2018,
  name = "año 2018"
)

val year2019 = Scatter(
  SeqMonths,
  Seq2019,
  name = "año 2019"
)

val year2020 = Scatter(
  SeqMonths,
  Seq2020,
  name = "año 2020"
)

val year2021 = Scatter(
  SeqMonths,
  Seq2021,
  name = "año 2021"
)

val year2022 = Scatter(
  SeqMonths,
  Seq2022,
  name = "año 2022"
)



val dataToPlot = Seq(year2010, year2011, year2012, year2013, year2014, year2015, year2016, 
                     year2017, year2018, year2019, year2020, year2021, year2022)

plot(dataToPlot)

### Representacion de las temperaturas máximas en diferentes meses por años (Parquet con particion)

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

In [ ]:
val dataRead = spark.read.parquet("D:/TFGAlvaroSanchez/data/monthParquet/*")

dataRead
    .withColumn("año", func.year($"fecha"))
    .repartition($"año").write.mode("overwrite").partitionBy("año").parquet("D:/TFGAlvaroSanchez/data/monthParquetPartition/")

In [ ]:
val data = spark.read.parquet("D:/TFGAlvaroSanchez/data/monthParquetPartition/").na.drop()

In [ ]:
def obtainData(year : Int) : Seq[Double] = {
    data
        .filter($"año" === year)
        .withColumn("ta_max", func.split($"ta_max", "\\(")(0).cast(IntegerType))
        .groupBy($"fecha")
        .agg(func.avg($"ta_max").alias("ta_max"))
        .withColumn("mes", func.month($"fecha"))
        .orderBy("mes")
        .map(row => row.getDouble(1)).collect().toSeq
}

In [ ]:
val seqMonths = data
    .filter($"año" === 2010)
    .groupBy($"fecha")
    .agg(func.avg($"ta_max").alias("ta_max"))
    .select(func.month($"fecha").alias("mes"))
    .orderBy($"mes")
    .map(row => row.getInt(0)).collect().toSeq
val seq2010 = obtainData(2010)
val seq2011 = obtainData(2011)
val seq2012 = obtainData(2012)
val seq2013 = obtainData(2013)
val seq2014 = obtainData(2014)
val seq2015 = obtainData(2015)
val seq2016 = obtainData(2016)
val seq2017 = obtainData(2017)
val seq2018 = obtainData(2018)
val seq2019 = obtainData(2019)
val seq2020 = obtainData(2020)
val seq2021 = obtainData(2021)
val seq2022 = obtainData(2022)

In [ ]:
val year2010= Scatter(
  seqMonths,
  seq2010,
  name = "año 2010"
)

val year2011 = Scatter(
  seqMonths,
  seq2011,
  name = "año 2011"
)

val year2012 = Scatter(
  seqMonths,
  seq2012,
  name = "año 2012"
)

val year2013 = Scatter(
  seqMonths,
  seq2013,
  name = "año 2013"
)

val year2014 = Scatter(
  seqMonths,
  seq2014,
  name = "año 2014"
)

val year2015 = Scatter(
  seqMonths,
  seq2015,
  name = "año 2015"
)

val year2016 = Scatter(
  seqMonths,
  seq2016,
  name = "año 2016"
)

val year2017 = Scatter(
  seqMonths,
  seq2017,
  name = "año 2017"
)

val year2018 = Scatter(
  seqMonths,
  seq2018,
  name = "año 2018"
)

val year2019 = Scatter(
  seqMonths,
  seq2019,
  name = "año 2019"
)

val year2020 = Scatter(
  seqMonths,
  seq2020,
  name = "año 2020"
)

val year2021 = Scatter(
  seqMonths,
  seq2021,
  name = "año 2021"
)

val year2022 = Scatter(
  seqMonths,
  seq2022,
  name = "año 2022"
)



val dataToPlot = Seq(year2010, year2011, year2012, year2013, year2014, year2015, year2016, 
                     year2017, year2018, year2019, year2020, year2021, year2022)

plot(dataToPlot)

### Numero de meses mas calurosos agrupados por años (Se compara mes a mes las temperaturas max)

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells

repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

In [ ]:
import org.apache.spark.sql.expressions.Window

val data = spark.read.parquet("D:/TFGAlvaroSanchez/data/monthParquet/*").na.drop()

val window = Window.partitionBy("mes").orderBy($"ta_max".desc)

val dataWindow = data
    .withColumn("ta_max", func.split($"ta_max", "\\(")(0).cast(DoubleType))
    .groupBy($"fecha")
    .agg(func.avg($"ta_max").alias("ta_max"))
    .select(func.month($"fecha").alias("mes"), func.year($"fecha").alias("año"), $"ta_max")
    .withColumn("dense_rank", func.dense_rank().over(window))
    .filter($"dense_rank" === 1)

In [ ]:
def countByYear(year : Int) : Long = {
    dataWindow
        .filter($"año" === year)
        .count()
}
 
val year2010 = countByYear(2010)
val year2011 = countByYear(2011)
val year2012 = countByYear(2012)
val year2013 = countByYear(2013)
val year2014 = countByYear(2014)
val year2015 = countByYear(2015)
val year2016 = countByYear(2016)
val year2017 = countByYear(2017)
val year2018 = countByYear(2018)
val year2019 = countByYear(2019)
val year2020 = countByYear(2020)
val year2021 = countByYear(2021)
val year2022 = countByYear(2022)

In [ ]:
dataWindow
    .withColumn("temperatura maxima", func.round($"ta_max", 2))
    .orderBy($"mes".asc)
    .select($"mes", $"año", $"temperatura maxima")
    .show()

val dataToPlot = Seq(
  Bar(
    Seq(2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022),
    Seq(year2010, year2011, year2012, year2013, year2014, year2015, year2016, 
        year2017, year2018, year2019, year2020, year2021, year2022),
    marker = Marker(
        color = Color.RGB(255, 174, 0),
        opacity = 0.6,
        line = Line(
            color = Color.RGB(189, 129, 0),
              width = 1.5
        )
    )
  )
)

plot(dataToPlot)

### Años/meses con olas de calor (30º durante >= 3 dias) Utilizando datos mensuales

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

In [ ]:
val data = spark.read.parquet("D:/TFGAlvaroSanchez/data/monthParquet/*").na.drop()

In [ ]:
val groupData = data
    .filter($"nt_30" >= 3)
    .withColumn("año", func.year($"fecha"))
    .withColumn("ta_max", func.split($"ta_max", "\\(")(0).cast(IntegerType))
    .groupBy($"año")
    .agg(func.avg($"nt_30").alias("nt_30"), func.avg($"ta_max").alias("ta_max"))
    .select($"año", func.round($"nt_30", 2) as "dias al mes con >=30º", func.round($"ta_max", 2) as "temperatura media max")
    .orderBy($"año")

groupData.show()

In [ ]:
val SeqYears = groupData.map(row => row.getInt(0)).collect().toSeq
val SeqDays = groupData.map(row => row.getDouble(1)).collect().toSeq
val SeqTemperature = groupData.map(row => row.getDouble(2).toInt).collect().toSeq

In [ ]:
val dataToPlot = Scatter(
    SeqYears,
    SeqDays,
    mode = ScatterMode(ScatterMode.Markers),
    marker = Marker(
        color = Seq(Color.RGB(251, 212, 68), Color.RGB(255, 223, 106), Color.RGB(251, 212, 68), Color.RGB(255, 223, 106),
                    Color.RGB(251, 212, 68), Color.RGB(251, 212, 68), Color.RGB(220, 170, 1), Color.RGB(251, 212, 68), 
                    Color.RGB(251, 212, 68), Color.RGB(220, 170, 1), Color.RGB(251, 212, 68), Color.RGB(220, 170, 1),
                    Color.RGB(220, 143, 1)),
        size = SeqTemperature
    )
)

val aux = Seq(dataToPlot)
plot(aux)

### Temperatura media por año y su relacion con las lluvias por año

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

In [ ]:
val data = spark.read.parquet("D:/TFGAlvaroSanchez/data/monthParquet/*").na.drop()

In [ ]:
val groupData = data
    .withColumn("año", func.year($"fecha"))
    .groupBy($"año")
    .agg(func.avg($"tm_mes").alias("tm_mes"), func.avg($"p_mes").alias("p_mes"))
    .orderBy($"año")

In [ ]:
groupData.first().fieldIndex("año")
groupData.first().fieldIndex("tm_mes")
groupData.first().fieldIndex("p_mes")

In [ ]:
val SeqYears = groupData.map(row => row.getInt(0)).collect().toSeq
val SeqTemperature = groupData.map(row => row.getDouble(1)).collect().toSeq
val SeqPrecipitation = groupData.map(row => row.getDouble(2)).collect().toSeq

In [ ]:
val dataTemperature = Scatter(
    SeqYears,
    SeqTemperature,
    name = "Temperatura media (ºC)",
    marker = Marker(
        color = Color.RGB(247, 112, 0)
    )
)

val dataPrecipitation = Scatter(
    SeqYears,
    SeqPrecipitation,
    name = "Precipitacion media (mm)",
    marker = Marker(
        color = Color.RGB(0, 187, 247)
    )
)

val layout1 = Layout(
    title = "Precipitacion y temperatura media anual"
)

val layout2 = Layout(
    title = "Temperatura media anual"
)

val dataToPlot = Seq(dataTemperature, dataPrecipitation)
plot(dataToPlot, layout1)

plot(Seq(dataTemperature), layout2)

### Lectura de datos por dias y transformacion a Parquet

In [ ]:
val schema = StructType(
    Array(
        StructField("fecha", StringType, true), 
        StructField("indicativo", StringType, true),
        StructField("nombre", StringType, true),
        StructField("provincia", StringType, true),
        StructField("altitud", StringType, true),
        StructField("tmed", StringType, true),
        StructField("prec", StringType, true),
        StructField("tmin", StringType, true),
        StructField("horatmin", StringType, true),
        StructField("tmax", StringType, true),
        StructField("horatmax", StringType, true),
        StructField("dir", StringType, true),
        StructField("velmedia", StringType, true),
        StructField("racha", StringType, true),
        StructField("horaracha", StringType, true),
        StructField("sol", StringType, true),
        StructField("presMax", StringType, true),
        StructField("horaPresMax", StringType, true),
        StructField("presMin", StringType, true),
        StructField("horaPresMin", StringType, true)
    )
)

In [ ]:
val allData = spark.read.schema(schema).json("D:/TFGAlvaroSanchez/data/dayJSONLine/*.json")
    .withColumn("fecha", $"fecha".cast(DateType))
    .withColumn("altitud", $"altitud".cast(IntegerType))
    .withColumn("tmed", func.regexp_replace($"tmed", ",", ".").cast(DoubleType))
    .withColumn("prec", func.regexp_replace($"prec", ",", ".").cast(DoubleType))
    .withColumn("tmin", func.regexp_replace($"tmin", ",", ".").cast(DoubleType))
    .withColumn("tmax", func.regexp_replace($"tmax", ",", ".").cast(DoubleType))
    .withColumn("dir", $"dir".cast(IntegerType))
    .withColumn("velmedia", func.regexp_replace($"velmedia", ",", ".").cast(DoubleType))
    .withColumn("racha", func.regexp_replace($"racha", ",", ".").cast(DoubleType))
    .withColumn("sol", func.regexp_replace($"sol", ",", ".").cast(DoubleType))
    .withColumn("presMax", func.regexp_replace($"presMax", ",", ".").cast(DoubleType))
    .withColumn("horaPresMax", $"horaPresMax".cast(IntegerType))
    .withColumn("presMin", func.regexp_replace($"presMin", ",", ".").cast(DoubleType))
    .withColumn("horaPresMin", $"horaPresmin".cast(IntegerType))

In [ ]:
allData.write.format("parquet").partitionBy("indicativo").mode("overwrite").save("D:/TFGAlvaroSanchez/data/dayParquet/")

### Años con olas de calor Utilizando datos diarios

In [ ]:
val data = spark.read.parquet("D:/TFGAlvaroSanchez/data/dayParquet/")
val stations = spark.read.option("delimiter", ";").csv("D:/TFGAlvaroSanchez/data/aemetID.csv")
    .toDF("provincia", "indicativo", "ubicacion")

In [ ]:
import org.apache.spark.sql.expressions.Window

val window = Window.partitionBy($"indicativo", $"año").orderBy($"fecha")

In [ ]:
val auxData = data
    .filter(!func.isnull($"tmax"))
    .withColumn("año", func.year($"fecha"))
    .withColumn("lag", func.lag($"tmax", 1).over(window))
    .withColumn("start", $"tmax" >= 40 && $"lag" < 40)
    .select($"indicativo", $"fecha", $"año", $"tmax", $"start")

In [ ]:
val secuenceData = auxData
    .withColumn("id", func.sum(func.when($"start", 1).otherwise(null)).over(window))
    .withColumn("id", func.when($"tmax" < 40, null).otherwise($"id"))
    .filter(!func.isnull($"id"))
    .select($"indicativo", $"fecha", $"año", $"tmax", $"id")

secuenceData.show()

In [ ]:
val results = secuenceData
    .groupBy($"indicativo", $"año", $"id")
    .agg(func.count($"id").alias("dias"), func.avg($"tmax"), func.max($"tmax"), func.min($"tmax"))
    .filter($"dias" > 3)
    .join(stations, "indicativo")
    .select($"ubicacion", $"provincia", $"año", $"dias", func.round($"avg(tmax)", 2).alias("avg(tmax)"), 
            $"max(tmax)", $"min(tmax)")

In [ ]:
results
    .orderBy($"ubicacion", $"año")
    .show()

In [ ]:
results
    .groupBy($"provincia", $"año")
    .agg(func.count($"provincia"), func.avg($"dias"), func.avg($"avg(tmax)"), func.max($"max(tmax)"), func.min($"min(tmax)"))
    .select($"provincia", $"año", $"count(provincia)".alias("nº de olas de calor"), 
            $"avg(dias)".alias("duracion media"), $"avg(avg(tmax))".alias("avg(tmax)"), 
            $"max(max(tmax))".alias("max(tmax)"), $"min(min(tmax))".alias("min(tmax)"))
    .orderBy($"provincia", $"año".asc)
    .show()

In [ ]:
results
    .groupBy($"año")
    .agg(func.count($"año"))
    .orderBy($"año")
    .show()

In [ ]:
val auxResults = results
    .groupBy($"provincia")
    .agg(func.count($"provincia").alias("nº olas de calor"))

val totalOlas = auxResults
    .agg(func.sum($"nº olas de calor"))
    .first().getLong(0)


auxResults
    .withColumn("% olas de calor", func.round($"nº olas de calor" / totalOlas * 100, 2))
    .orderBy($"provincia")
    .show()

### Años con olas de calor Utilizando datos diarios (haciendo uso de row number)

In [3]:
import org.apache.spark.sql.expressions.Window

val data = spark.read.parquet("D:/TFGAlvaroSanchez/data/dayParquet/")
val stations = spark.read.option("delimiter", ";").csv("D:/TFGAlvaroSanchez/data/aemetID.csv")
    .toDF("provincia", "indicativo", "ubicacion")

val window = Window.partitionBy($"indicativo", $"año").orderBy($"fecha")

val results = data
    .filter(!func.isnull($"tmax") && $"tmax" >= 40)
    .withColumn("año", func.year($"fecha"))
    .withColumn("n_fila", func.row_number().over(window))
    .withColumn("id", func.expr("date_sub(fecha, n_fila)"))
    .groupBy($"indicativo", $"año", $"id")
    .agg(func.count($"id").alias("dias"), func.avg($"tmax"), func.max($"tmax"), func.min($"tmax"))
    .filter($"dias" > 3)
    .join(stations, "indicativo")
    .select($"ubicacion", $"provincia", $"año", $"dias", func.round($"avg(tmax)", 2).alias("avg(tmax)"), 
            $"max(tmax)", $"min(tmax)")

parquet at cmd2.sc:3

272 / 272

parquet at cmd2.sc:3

272 / 272

parquet at cmd2.sc:3

...

csv at cmd2.sc:4

1 / 1

csv at cmd2.sc:4

1 / 1

import org.apache.spark.sql.expressions.Window


data: DataFrame = [fecha: date, nombre: string ... 18 more fields]
stations: DataFrame = [provincia: string, indicativo: string ... 1 more field]
window: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@262ee88c
results: DataFrame = [ubicacion: string, provincia: string ... 5 more fields]

In [4]:
val resultsSave = results
    .groupBy($"provincia", $"año")
    .agg(func.count($"provincia"), func.avg($"dias"), func.avg($"avg(tmax)"), func.max($"max(tmax)"), func.min($"min(tmax)"))
    .select($"provincia", $"año", $"count(provincia)".alias("nº de olas de calor"), 
            $"avg(dias)".alias("duracion media"), $"avg(avg(tmax))".alias("avg(tmax)"), 
            $"max(max(tmax))".alias("max(tmax)"), $"min(min(tmax))".alias("min(tmax)"))
    
resultsSave
    .withColumnRenamed("nº de olas de calor", "nOlasCalor")
    .withColumnRenamed("duracion media", "duracionMedia")
    .withColumnRenamed("avg(tmax)", "avgTmax")
    .withColumnRenamed("max(tmax)", "maxTmax")
    .withColumnRenamed("min(tmax)", "minTmax")
    .write.format("parquet").partitionBy("provincia").mode("overwrite").save("D:/TFGAlvaroSanchez/data/resultadoOlasCalor/")

run at ThreadPoolExecutor.java:1128

1 / 1

run at ThreadPoolExecutor.java:1128

1 / 1

save at cmd3.sc:14

...

save at cmd3.sc:14

64 / 64

save at cmd3.sc:14

64 / 64

resultsSave: DataFrame = [provincia: string, año: int ... 5 more fields]

In [4]:
results.show()

run at ThreadPoolExecutor.java:1128

1 / 1

show at cmd3.sc:1

90 / 90

show at cmd3.sc:1

...

show at cmd3.sc:1

4 / 4

show at cmd3.sc:1

5 / 5

+--------------------+-----------+----+----+---------+---------+---------+
|           ubicacion|  provincia| año|dias|avg(tmax)|max(tmax)|min(tmax)|
+--------------------+-----------+----+----+---------+---------+---------+
|                Loja|    Granada|2021|   4|    44.28|     45.6|     42.3|
|Navalmoral de la ...|    Caceres|2015|   4|     40.7|     41.2|     40.3|
|Navalmoral de la ...|    Caceres|2015|   4|    40.95|     42.5|     40.3|
|                Baza|    Granada|2007|   4|    40.85|     41.3|     40.1|
|           Antequera|     Malaga|2017|   5|    42.68|     44.4|     40.1|
|              Merida|    Badajoz|2017|   5|    42.46|     44.0|     40.5|
|La Roda de Andalucia|    Sevilla|2015|   5|    41.14|     43.2|     40.0|
|La Roda de Andalucia|    Sevilla|2015|   4|     40.6|     41.1|     40.2|
|               ecija|    Sevilla|2007|   6|    42.08|     43.2|     41.0|
|   Madrid Aeropuerto|     Madrid|2022|   4|     41.0|     42.2|     40.5|
|              Toledo|   

In [5]:
results
    .withColumnRenamed("dias", "duracion (dias)")
    .withColumnRenamed("avg(tmax)", "temperatura media")
    .withColumnRenamed("max(tmax)", "temperatura maxima")
    .withColumnRenamed("min(tmax)", "temperatura minima")
    .orderBy($"ubicacion", $"año")
    .show()

run at ThreadPoolExecutor.java:1128

1 / 1

show at cmd4.sc:6

90 / 90

show at cmd4.sc:6

64 / 64

+---------+---------+----+---------------+-----------------+------------------+------------------+
|ubicacion|provincia| año|duracion (dias)|temperatura media|temperatura maxima|temperatura minima|
+---------+---------+----+---------------+-----------------+------------------+------------------+
|   Alajar|   Huelva|2018|              4|            41.65|              42.5|              40.7|
|  Alcaniz|   Teruel|2019|              4|            40.95|              41.2|              40.8|
|  Andujar|     Jaen|2008|              5|            41.32|              42.7|              40.6|
|  Andujar|     Jaen|2009|              4|             41.5|              42.7|              40.4|
|  Andujar|     Jaen|2009|              6|            41.82|              42.6|              40.5|
|  Andujar|     Jaen|2010|              7|            41.39|              42.7|              40.4|
|  Andujar|     Jaen|2010|              4|            42.35|              43.1|              40.9|
|  Andujar

In [6]:
results
    .groupBy($"año")
    .agg(func.count($"año").alias("nº olas de calor"))
    .orderBy($"año")
    .show()

run at ThreadPoolExecutor.java:1128

1 / 1

run at ThreadPoolExecutor.java:1128

1 / 1

show at cmd5.sc:4

...

show at cmd5.sc:4

64 / 64

show at cmd5.sc:4

64 / 64

+----+----------------+
| año|nº olas de calor|
+----+----------------+
|2007|               6|
|2008|               1|
|2009|               9|
|2010|               6|
|2012|               6|
|2013|               4|
|2014|               1|
|2015|              28|
|2016|              11|
|2017|              36|
|2018|              23|
|2019|              10|
|2020|              11|
|2021|              24|
|2022|              68|
+----+----------------+

